In [1]:
import rpt_parser
import pandas as pd

In [73]:
class OpenSTAParser:
    """

    Methodology:
    1. Read metadata on the type of file and run this is.
    2. Identify the boundary lines of the table data through regex matching.
    3. Read in table data of the timing performance.
    4. Understand the relevant timing and data parameters.
    5. Extract the relevant timing and data parameters from the
    """
    
    def __init__(
            self,
            file_address="25-rcx_sta.rpt",
    ):
        self.file_address = file_address
        
        
        # Internal
        self.file_lines_raw = None
        self.timing_data = None
        self.file_lines_raw = None
        self.file_lines_data = None
        self.meta_data = pd.DataFrame()
        
        self.read_file_meta_data()
        self.extract_frame_meta_data()
        
        self.frame_timing_data = {}
        for frame_id in range(self.maximum_frame_amount):
            self.frame_timing_data[frame_id] = self.extract_timing_data(frame_id=frame_id)

    def read_file_meta_data(self):
        self.file = open(self.file_address, "r")
        self.file_lines_raw = self.file.readlines()
        self.file_lines_data = pd.DataFrame({"lines": self.file_lines_raw})
        self.configure_frame_id()
        self.configure_timing_data_rows()
    
    def configure_frame_id(self):
        self.file_lines_data["delimiters_line"] = self.file_lines_data.lines.str.contains("==========")
        self.file_lines_data["timing_data_line"] = self.file_lines_data.lines.str.contains("---------")
        frame_data = []
        frame_id_counter = 0
        parity_counter = 1
        row = 0
        for delimiter in self.file_lines_data["delimiters_line"]:
            if (delimiter):
                if (parity_counter % 2):
                    frame_id_counter += 1
                    parity_counter = 0
                else:
                    parity_counter += 1
            frame_data.append(frame_id_counter-1)
            row += 1
        self.file_lines_data["frame_id"] = frame_data
        self.maximum_frame_amount = frame_id_counter
    
    def configure_timing_data_rows(self):
        self.frame_meta_data = {}
        for frame in range(self.maximum_frame_amount):
            timing_rows_index = self.file_lines_data.index[self.file_lines_data['timing_data_line'] & (self.file_lines_data['frame_id'] == frame)].tolist()
            if len(timing_rows_index) >= 1:
                
                self.frame_meta_data[frame] = {
                    "start_index": timing_rows_index[0],
                    "end_index": timing_rows_index[-1],
                    "start_rows_skip": timing_rows_index[0] + 1,
                    "end_rows_skip": len(self.file_lines_data) - timing_rows_index[-1],
                }
            else:
                self.frame_meta_data[frame] = {
                    "start_index": 0,
                    "end_index": 0,
                    "start_rows_skip": 0,
                    "end_rows_skip": 0,
                }
        return self.frame_meta_data
    
    def extract_frame_meta_data(self):
        # Extract Frame Metadata, note that these operations can be performed dataset wise.
        self.file_lines_data["start_point_line"] = self.file_lines_data.lines.str.contains("Startpoint")
        self.file_lines_data["end_point_line"] = self.file_lines_data.lines.str.contains("Endpoint")
        self.file_lines_data["path_group_line"] = self.file_lines_data.lines.str.contains("Path Group")
        self.file_lines_data["path_type_line"] = self.file_lines_data.lines.str.contains("Path Type")
                
        self.start_point_name = self.file_lines_data.lines[self.file_lines_data.start_point_line].str.extract(r'((?<=Startpoint:\s).*?(?=\s\())')
        self.end_point_name = self.file_lines_data.lines[self.file_lines_data.end_point_line].str.extract(r'((?<=Endpoint:\s).*?(?=\s\())')
        self.path_group_name = self.file_lines_data.lines[self.file_lines_data.path_group_line].str.extract(r'((?<=Path Group:\s).*)')
        self.path_type_name = self.file_lines_data.lines[self.file_lines_data.path_type_line].str.extract(r'((?<=Path Type:\s).*)')
        self.timing_data_start = self.file_lines_data.lines[self.file_lines_data.path_type_line].str.extract(r'((?<=Path Type:\s).*)')

    def extract_timing_data(self, frame_id=0):
        timing_data = pd.read_fwf(self.file_address,
                                  colspecs=[(0,6), (6,14), (14,22), (22,30), (30,38), (38,40) ,(40,100)],
                                  skiprows=self.frame_meta_data[frame_id]["start_rows_skip"], 
                                  skipfooter=self.frame_meta_data[frame_id]["end_rows_skip"], 
                                  names=["Fanout", "Cap", "Slew", "Delay", "Time", "Direction", "Description"])
        timing_data["net"] = timing_data["Description"].str.extract(r'\(([^()]+)\)')
        return timing_data
        
    def propagation_delay(self, 
                          net_in="in",
                          net_out="out"):
        return self.timing_data[self.timing_data.net == net_out].Time.values - self.timing_data[self.timing_data.net == net_in].Time.values
    

In [75]:
a = OpenSTAParser()
a.frame_timing_data[1]

0
1


,Fanout,Cap,Slew,Delay,Time,Direction,Description,net
0,======,========,========,========,========,==,===================================,NaN
1,report,_checks,-unconst,rained,NaN,NaN,NaN,NaN
2,======,========,========,========,========,==,====================================,NaN
3,Startp,oint: in,(input,port clo,cked by,__,VIRTUAL_CLK__),NaN
4,Endpoi,nt: out,(output,port clo,cked by,__,VIRTUAL_CLK__),NaN
5,Path G,roup: __,VIRTUAL_,CLK__,NaN,NaN,NaN,NaN
6,Path T,ype: max,NaN,NaN,NaN,NaN,NaN,NaN
7,Fanout,Cap,Slew,Delay,Time,NaN,Description,NaN
8,------,--------,--------,--------,--------,--,-------------------------------------,NaN
9,NaN,NaN,0.00,0.00,0.00,NaN,clock __VIRTUAL_CLK__ (rise edge),rise edge


In [13]:
a.file_lines_data

,lines,delimiters_line,frame_id
0,\n,False,-1
1,==============================================...,True,0
2,report_checks -unconstrained\n,False,0
3,==============================================...,True,0
4,Startpoint: in (input port clocked by __VIRTUA...,False,0
5,Endpoint: out (output port clocked by __VIRTUA...,False,0
6,Path Group: __VIRTUAL_CLK__\n,False,0
7,Path Type: max\n,False,0
8,\n,False,0
9,Fanout Cap Slew Delay Time Descr...,False,0


In [195]:
a.file_lines_data.lines[a.file_lines_data.start_point_line].str.extract(r'((?<=Startpoint:\s).*?(?=\s\())')
a.file_lines_data.lines[a.file_lines_data.end_point_line].str.extract(r'((?<=Endpoint:\s).*?(?=\s\())')
a.file_lines_data.lines[a.file_lines_data.path_group_line].str.extract(r'((?<=Path Group:\s).*)')
a.file_lines_data.lines[a.file_lines_data.path_type_line].str.extract(r'((?<=Path Type:\s).*)')

AttributeError: 'DataFrame' object has no attribute 'start_point_line'

In [3]:
import os

example_file_address = os.path.dirname(rpt_parser.__file__) + "/../tests/25-rcx_sta.rpt"

a = rpt_parser.ParserRPT(
    file_address=example_file_address,
)
# 

['\n', '===========================================================================\n', 'report_checks -unconstrained\n', '============================================================================\n', 'Startpoint: in (input port clocked by __VIRTUAL_CLK__)\n', 'Endpoint: out (output port clocked by __VIRTUAL_CLK__)\n', 'Path Group: __VIRTUAL_CLK__\n', 'Path Type: max\n', '\n', 'Fanout     Cap    Slew   Delay    Time   Description\n', '-----------------------------------------------------------------------------\n', '                  0.00    0.00    0.00   clock __VIRTUAL_CLK__ (rise edge)\n', '                          0.00    0.00   clock network delay (ideal)\n', '                          2.00    2.00 v input external delay\n', '                  0.01    0.01    2.01 v in (in)\n', '     1    0.00                           in (net)\n', '                  0.01    0.00    2.01 v input1/A (sky130_fd_sc_hd__clkbuf_1)\n', '                  0.05    0.09    2.10 v input1/X (sky130_fd_s

In [48]:
match = '===========================================================================\n'
matches = []
for line in a.file_lines:
    if match in line:
        matches.append(match)
matches

['===========================================================================\n',
 '===========================================================================\n',
 '===========================================================================\n',
 '===========================================================================\n']

0     False
1     False
2     False
3     False
4      True
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
dtype: bool

In [4]:
text_table_joined = "".join(a.file_lines[9:28])

In [5]:
table_file = open("test.csv", "w")
table_file.write(text_table_joined)
table_file.close()

In [10]:
data = pd.read_fwf("25-rcx_sta.rpt",
            colspecs=[(0,6), (6,14), (14,22), (22,30), (30,38), (38,40) ,(40,100)],
            skiprows=11, 
            skipfooter=12,
            names=["Fanout", "Cap", "Slew", "Delay", "Time", "Direction", "Description"])
data["net"] = data["Description"].str.extract(r'\(([^()]+)\)')
data

,Fanout,Cap,Slew,Delay,Time,Direction,Description,net
0,NaN,NaN,0.00,0.00,0.00,NaN,clock __VIRTUAL_CLK__ (rise edge),rise edge
1,NaN,NaN,NaN,0.00,0.00,NaN,clock network delay (ideal),ideal
2,NaN,NaN,NaN,2.00,2.00,v,input external delay,NaN
3,NaN,NaN,0.01,0.01,2.01,v,in (in),in
4,1.0,0.00,NaN,NaN,NaN,NaN,in (net),net
5,NaN,NaN,0.01,0.00,2.01,v,input1/A (sky130_fd_sc_hd__clkbuf_1),sky130_fd_sc_hd__clkbuf_1
6,NaN,NaN,0.05,0.09,2.10,v,input1/X (sky130_fd_sc_hd__clkbuf_1),sky130_fd_sc_hd__clkbuf_1
7,1.0,0.01,NaN,NaN,NaN,NaN,net1 (net),net
8,NaN,NaN,0.05,0.00,2.10,v,_0_/A (sky130_fd_sc_hd__inv_2),sky130_fd_sc_hd__inv_2
9,NaN,NaN,0.03,0.05,2.15,^,_0_/Y (sky130_fd_sc_hd__inv_2),sky130_fd_sc_hd__inv_2


array([0.35])